In [1]:
import json
import subprocess

In [2]:
with open("../data/formatted-data/test_data_formatted.json", "r") as f:
    test_data = json.load(f)
print(test_data[0:5])

[{'instruction': 'Based off this context, output whether the question implies entailment or non-entailment', 'input': 'Context: Harish is shorter than Suresh. Manish is taller than Raghu. Harish is taller than Anil. , Question: Is Harish taller than Manish ?', 'output': 'not entailment - unknown\n'}, {'instruction': 'Based off this context, output whether the question implies entailment or non-entailment', 'input': 'Context: Jayesh is taller than Nandu. Ramesh is shorter than Nandu. Satish is shorter than Subodh. Vinod is taller than Jayesh. , Question: Is Nandu shorter than Vinod ?', 'output': 'not entailment - contradiction\n'}, {'instruction': 'Based off this context, output whether the question implies entailment or non-entailment', 'input': 'Context: Tom is shorter than Mary. Mary is shorter than Sara but taller than Tom. Mike is taller than John but shorter than Katy. , Question: Is Tom taller than Tom ?', 'output': 'not entailment - contradiction\n'}, {'instruction': 'Based off 

In [ ]:
import csv
checkpoint_dir = "../out4/lora_weights/mistral7B-finetuned/final/"
output_csv_file = 'logic-games-llms/data/finetuned_eval_responses.csv'

for i, d in enumerate(test_data):
    prompt = f"{d['input']} {d['instruction']}"
    command = [
        "litgpt", "generate", "base",
        "--precision", "bf16-true",
        "--checkpoint_dir", checkpoint_dir,
        "--prompt", prompt
    ]
    result = subprocess.run(command, capture_output=True, text=True)
    if result.returncode != 0:
        print(f"Command failed with error: {result.stderr}")
        continue
    str_out = result.stdout
    try:
      str_out = str_out.split("### Response:")[1].strip()
    except:
      print(f"Expected marker '### Response:' not found in output for iteration {i}.")
      str_out = None
    # Write the prompt and its corresponding output to the CSV file
    with open(output_csv_file, 'a', newline='', encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile)
        # Check if file is empty to write headers
        if csvfile.tell() == 0:
            csvwriter.writerow(['Index', 'Prompt', 'Output'])
        csvwriter.writerow([i, prompt, str_out])

    if i % 100 == 0:
      print(f"{i}/{len(test_data)}")